# **Chapter 11 심층 신경망 훈련**

## **11.1 그래디언트 소실과 폭주 문제**

그래디언트 소실: 알고리즘이 하위층으로 진행됨에 따라 그래디언트가 점점 작아지는 것(-> 결국 경사 하강법이 하위층의 연결 가중치를 실제 변경되지 않은 채로 둔다면 훈련이 좋은 솔루션으로 수렴되지 않는다.)

그래디언트 폭주: 그래디언트가 점점 커져 여러 개의 층이 비정상적으로 큰 가중치로 갱신되어 알고리즘이 발산하는 것

### 11.1.1 세이비어 초기화와 He 초기화

예측을 할 때(정방향), 그래디언트를 역전파할 때(역방향) 신호가 적절하게 흐르게 하기 위해서는 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다. 그리고 역방향에서 층을 통과하기 전과 후의 그래디언트 분산이 동일해야 한다. 연결 가중치를 세이비어 초기화 전략으로 무작위 초기화하면 그 두가지를 충족하여 잘 작동하게 할 수 있다. 또한 세이비어 초기화 전략을 사용하면 훈련 속도를 상당히 높일 수 있다. 

ReLU 활성화 함수나 ReLU의 변종 활성화 함수를 위한 초기화 전략을 He 초기화라고 한다.

In [0]:
%tensorflow_version 1.x
import tensorflow as tf

In [0]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [4]:
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X,n_hidden1,activation=tf.nn.relu,kernel_initializer=he_init,name="hidden1")

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


tf.lyaer.dense() 함수는 기본적으로 세이비어 초기화를 사용한다. variance_scaling_initializer() 함수를 사용하면 He 초기화 방식으로 바꿀 수 있다.

### 11.1.2 수렴하지 않는 활성화 함수

ReLU 함수는 특정 양숫값에 수렴하지 않으며 계산이 빠르다는 장점이 있지만, 완벽하지는 않다. 죽은 ReLU 문제 때문이다. 훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는다는 의미로 죽었다고 말한다. 훈련 도중 뉴런의 가중치가 바뀌어 가중치 합이 음수가 되면 그 다음부터 0을 출력하기 시작하는데, ReLU 함수는 입력이 음수면 그래디언트가 0이 되기 때문에 이런 일이 생기면 뉴런이 다시 살아나기 어렵다. 

이러한 문제 때문에 LeakyReLU와 같은 ReLU 함수의 변종을 사용하게 된다. (식은 교재 참고) 하이퍼파라미터 alpha가 이 함수가 새는 정도를 결정한다. 새는 정도는 z<0일 때 이 함수의 기울기이며, 일반적으로 0.01로 설정한다. 이 기울기가 LeakyReLU를 절대 죽지 않게 만들어준다. LeakyReLU는 ReLU보다 항상 성능이 높다. 또 다른 변종 함수 RReLU는 훈련하는 동안 주어진 범위에서 alpha를 무작위로 선택하고 테스트 시에는 평균을 사용한다. 이는 훈련 세트의 과대적합 위험을 줄이는 규제의 역할을 하는 것으로 보이며, 대규모 이미지셋에서 ReLU보다 성능이 높다. 그러나 소규모 데이터셋에서는 훈련 세트에 과대적합될 위험이 있다. 

마지막으로 ELU라는 새로운 활성화 함수는 훈련 시간도 적게 들고 신경망의 테스트 세트 성능도 상당히 높다. (식은 교재 참고) 다른 모든 ReLU 변종의 성능을 앞질렀다. 주요 단점은 지수 함수를 사용하기 때문에 ReLU나 다른 변종들에 비해 계산이 느리다는 것이다.

In [0]:
hidden1 = tf.layers.dense(X,n_hidden1,activation=tf.nn.elu,name="hidden2")

dense() 함수를 호출할 때 activation 매개변수에 지정하기만 하면 ELU 활성화 함수를 이용할 수 있다.

In [0]:
def leaky_relu(z, name=None):
  return tf.maximum(0.01*z,z,name=name)

hidden1 = tf.layers.dense(X,n_hidden1,activation=leaky_relu,name="hidden3")

LeakyReLU는 텐서플로가 기본으로 제공하지는 않지만, 다음처럼 간단히 만들어 사용할 수 있다.

### 11.1.3 배치 정규화

배치 정규화는 그래디언트 소실과 폭주 문제 해결을 위한 기법이다. 더 일반적으로는, 훈련하는 동안 이전 층의 파라미터가 변함에 따라 각 층에 들어오는 입력의 분포가 변화되는 문제이다.(내부 공변량 변화 문제) 이 기법은 각 층에서 활성화 함수를 통과하기 전에 모델에 연산을 하나 추가한다. 단순하게 입력 데이터의 평균을 0으로 만들고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터로 결괏값의 스케일을 조정하고 이동시킨다. 이 연산으로 모델이 층마다 입력 데이터의 최적 스케일과 평균을 학습한다. 

입력 데이터의 평균을 0으로 만들고 정규화하려면 알고리즘은 평균과 표준편차를 추정해야 한다. 이를 위해 현재 미니배치에서 입력의 평균과 표준편차를 평가한다. 
테스트에서는 평균과 표준편차를 계산할 미니배치가 없으니 전체 훈련 세트의 평균과 표준편차를 대신 사용한다.

그러나 배치 정규화는 모델의 복잡도를 키우고, 실행 시간 면에서도 손해이다. 층마다 추가되는 계산이 신경망의 예측을 느려지게 하기 때문이다. 따라서 배치 정규화 사용 전에 ELU+He 초기화만으로 얼마나 잘 수행되는지 체크해보는 것이 좋다.

In [7]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",reuse=tf.AUTO_REUSE)
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2",reuse=tf.AUTO_REUSE)
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)


logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs",reuse=tf.AUTO_REUSE)
logits = tf.layers.batch_normalization(logits_before_bn, training=training,
                                       momentum=0.9)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


tf.layers.batch_normalization() 함수를 사용해 평균과 표준편차를 계산하고 스케일 조정과 이동을 위한 파라미터를 생성해 tf.nn.batch_normalization() 함수(입력값을 중앙에 정렬하고 정규화해주는 함수)에 전달하는 복잡한 과정을 쉽게 처리했다. 

training은 훈련하는 동안에는 True로 그 외에는 False로 설정하는데, 이를 이용해 tf.layers.batch_normalization() 함수가 현재 미니배치의 평균과 표준편차를 사용할지(훈련할 때) 또는 전체 훈련 세트에 대한 평균과 표준편차를 사용할지(테스트할 때) 지정한다. 

그 다음에는 tf.layers.dense()를 사용해 생성한 완전 연결 층과 tf.layers.batch_normalization() 함수를 사용해 생성한 배치 정규화 층이 번갈아가며 나온다. BN 알고리즘은 지수 감소를 사용해 이동 평균을 계산하기 때문에 momentum 매개변수가 필요하다. 적절한 모멘텀 값은 일반적으로 1에 가까운데(0.9, 0.99, 0.999...) 데이터셋이 크고 미니배치가 작을 경우 9를 더 넣어 1에 더 가깝게 한다. 

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization,
                              training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",reuse=True)
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2",reuse=True)
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs",reuse=True)
logits = my_batch_norm_layer(logits_before_bn)

partial() 함수를 사용해서 래퍼 함수를 생성했다. (매개변수의 기본값을 지정할 수 있다.)  그렇게 해서 코드 중복을 피했다.

In [9]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [13]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.288
1 검증 세트 정확도: 0.7942
2 검증 세트 정확도: 0.8798
3 검증 세트 정확도: 0.906
4 검증 세트 정확도: 0.9162
5 검증 세트 정확도: 0.9216
6 검증 세트 정확도: 0.9292
7 검증 세트 정확도: 0.9356
8 검증 세트 정확도: 0.938
9 검증 세트 정확도: 0.9414
10 검증 세트 정확도: 0.9456
11 검증 세트 정확도: 0.9472
12 검증 세트 정확도: 0.9476
13 검증 세트 정확도: 0.9534
14 검증 세트 정확도: 0.957
15 검증 세트 정확도: 0.9564
16 검증 세트 정확도: 0.9576
17 검증 세트 정확도: 0.959
18 검증 세트 정확도: 0.9624
19 검증 세트 정확도: 0.9612


MNIST 정확도가 그다지 좋지 않다. 배치 정규화와 ELU는 이런 얕은 신경망보다는 심층 신경망에서 큰 효과를 낸다.



### 11.1.4 그래디언트 클리핑

그래디언트 폭주 문제를 줄이는 쉬운 방법은 역전파될 때 일정 임곗값을 넘어서지 못하게 그래디언트를 그냥 단순히 잘라내는 것이다. 이는 순환 신경망에서 일반적으로 널리 사용되며, 그래디언트 클리핑이라고 부른다.

In [0]:
threshold = 1.0
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad,-threshold,threshold),var)for grad,var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

그래디언트를 계산하고 그것을 -1.0과 1.0 사이로 클리핑해서 적용했다.

## **11.2 미리 훈련된 층 재사용하기**

아주 큰 규모의 DNN의 경우 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아보고 그런 다음 그 신경망의 하위층을 재사용하는 것이 좋다. 이를 전이 학습이라고 한다. 

### 11.2.1 텐서플로 모델 재사용하기

In [0]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

원본 모델이 텐서플로를 사용해 훈련되었다면 간단하게 바로 복원해서 새로운 작업에 훈련시킬 수 있다. import_meta_graph() 함수를 사용해 기본 그래프에 연산을 적재했다. 이 함수는 Saver 객체를 반환하는데, 나중에 저장된 모델 파라미터를 불러올 때 사용한다.

In [0]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("GradientDescent")

훈련 대상인 연산과 텐서를 직접 지정했다. 필요한 연산을 get_tensor_by_name() 메서드와 get_operation_by_name() 메서드로 추출했다.

In [17]:
for op in tf.get_default_graph().get_operations():
  print(op.name)

X
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
dn

get_operations() 메서드로 모든 연산의 리스트를 확인해보았다.

In [0]:
for op in (X,y,accuracy,training_op):
  tf.add_to_collection("my_important_ops",op)

다른 사람을 위해 중요한 연산들을 모아놓은 컬렉션을 만들었다.

In [0]:
X,y,accuracy,training_op = tf.get_collection("my_important_ops")

다른 사람이 모델을 재사용하기 위해서는 위와 같은 코드를 입력하면 된다.

In [20]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    # 모델 훈련 계속하기...

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


세션을 시작하고 Saver 객체를 사용해 모델의 상태를 복원했다. 그 뒤로는 본인만의 데이터로 훈련을 계속해나가면 된다.

In [21]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 검증 세트 정확도: 0.9636
1 검증 세트 정확도: 0.9646
2 검증 세트 정확도: 0.9648
3 검증 세트 정확도: 0.9644
4 검증 세트 정확도: 0.9664
5 검증 세트 정확도: 0.9674
6 검증 세트 정확도: 0.968
7 검증 세트 정확도: 0.9666
8 검증 세트 정확도: 0.9656
9 검증 세트 정확도: 0.971
10 검증 세트 정확도: 0.9674
11 검증 세트 정확도: 0.9704
12 검증 세트 정확도: 0.971
13 검증 세트 정확도: 0.9708
14 검증 세트 정확도: 0.9692
15 검증 세트 정확도: 0.97
16 검증 세트 정확도: 0.9712
17 검증 세트 정확도: 0.9694
18 검증 세트 정확도: 0.9724
19 검증 세트 정확도: 0.9712


In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
learning_rate = 0.01
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [23]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 검증 세트 정확도: 0.9638
1 검증 세트 정확도: 0.9632
2 검증 세트 정확도: 0.9652
3 검증 세트 정확도: 0.9652
4 검증 세트 정확도: 0.9642
5 검증 세트 정확도: 0.965
6 검증 세트 정확도: 0.9686
7 검증 세트 정확도: 0.9684
8 검증 세트 정확도: 0.9682
9 검증 세트 정확도: 0.9688
10 검증 세트 정확도: 0.9704
11 검증 세트 정확도: 0.9716
12 검증 세트 정확도: 0.9672
13 검증 세트 정확도: 0.9704
14 검증 세트 정확도: 0.9708
15 검증 세트 정확도: 0.9722
16 검증 세트 정확도: 0.9718
17 검증 세트 정확도: 0.9712
18 검증 세트 정확도: 0.9714
19 검증 세트 정확도: 0.9714


In [0]:
reset_graph()

n_hidden4 = 20  # 새 층
n_outputs = 10  # 새 층

saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden3 = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/Relu:0")

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="new_hidden4")
new_logits = tf.layers.dense(new_hidden4, n_outputs, name="new_outputs")

with tf.name_scope("new_loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("new_eval"):
    correct = tf.nn.in_top_k(new_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("new_train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

일반적으로 하위층만 재사용한다. import_meta_graph()를 사용하면 전체 그래프를 로드하지만 필요하지 않은 부분은 무시하면 된다. 이 예에서는 학습된 3번째 층 위에 4번째 은닉층을 새로 추가했다(원래 4번째 층은 무시된다). 새로운 출력층도 추가하고 이 출력으로 손실을 계산하고 이를 최소화하기 위한 새로운 옵티마이저를 만들었다. 전체 그래프(원본 그래프 전체와 새로운 연산)를 저장할 새로운 Saver 객체를 만들고 새로운 모든 변수를 초기화할 초기화 연산도 해주었다.

In [25]:
with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = new_saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 검증 세트 정확도: 0.919
1 검증 세트 정확도: 0.9394
2 검증 세트 정확도: 0.949
3 검증 세트 정확도: 0.9524
4 검증 세트 정확도: 0.9552
5 검증 세트 정확도: 0.956
6 검증 세트 정확도: 0.9578
7 검증 세트 정확도: 0.961
8 검증 세트 정확도: 0.9614
9 검증 세트 정확도: 0.964
10 검증 세트 정확도: 0.965
11 검증 세트 정확도: 0.9656
12 검증 세트 정확도: 0.964
13 검증 세트 정확도: 0.967
14 검증 세트 정확도: 0.9678
15 검증 세트 정확도: 0.9678
16 검증 세트 정확도: 0.9696
17 검증 세트 정확도: 0.9674
18 검증 세트 정확도: 0.9696
19 검증 세트 정확도: 0.9706


새로운 모델을 훈련시켰다.

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # 재사용
n_hidden2 = 50  # 재사용
n_hidden3 = 50  # 재사용
n_hidden4 = 20  # 새로 만듦!
n_outputs = 10  # 새로 만듦!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # 재사용
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # 재사용
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # 재사용
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # 새로 만듦!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # 새로 만듦!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

필요한 부분만 재사용하고 나머지는 버렸다.

In [27]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # 정규표현식
restore_saver = tf.train.Saver(reuse_vars) # 1-3층 복원

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):                                       
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})   
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)                      

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 검증 세트 정확도: 0.9026
1 검증 세트 정확도: 0.9332
2 검증 세트 정확도: 0.9428
3 검증 세트 정확도: 0.947
4 검증 세트 정확도: 0.9524
5 검증 세트 정확도: 0.9532
6 검증 세트 정확도: 0.9556
7 검증 세트 정확도: 0.9588
8 검증 세트 정확도: 0.9588
9 검증 세트 정확도: 0.9606
10 검증 세트 정확도: 0.962
11 검증 세트 정확도: 0.962
12 검증 세트 정확도: 0.9638
13 검증 세트 정확도: 0.9658
14 검증 세트 정확도: 0.9664
15 검증 세트 정확도: 0.966
16 검증 세트 정확도: 0.9672
17 검증 세트 정확도: 0.9674
18 검증 세트 정확도: 0.9682
19 검증 세트 정확도: 0.9676


은닉층 1~3까지를 복원하고 새로운 모델을 저장했다. 그리고 세션을 열어 모델을 훈련시켰다.

### 1.2.2 다른 프레임워크의 모델 재사용하기

만약 모델이 다른 프레임워크로 훈련되어 있다면 수동으로 모델 파라미터를 읽어 들여 적절한 변수에 할당해야 한다.

In [0]:
reset_graph()

n_inputs = 2
n_hidden1 = 3

In [29]:
original_w = [[1., 2., 3.], [4., 5., 6.]] # 다른 프레임워크로부터 가중치를 로드
original_b = [7., 8., 9.]                 # 다른 프레임워크로부터 편향을 로드

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
# [...] 모델의 나머지 부분을 구성

# hidden1 변수의 할당 노드에 대한 핸들을 구한다
graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name("hidden1/kernel/Assign")
assign_bias = graph.get_operation_by_name("hidden1/bias/Assign")
init_kernel = assign_kernel.inputs[1]
init_bias = assign_bias.inputs[1]

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init, feed_dict={init_kernel: original_w, init_bias: original_b})
    # [...] 새 작업에 모델을 훈련시킵니다
    print(hidden1.eval(feed_dict={X: [[10.0, 11.0]]})) 

[[ 61.  83. 105.]]


또 다른 방법은 전용 할당 노드와 플레이스홀더를 만드는 것이다. 이 방법은 더 번거롭지만 하려는 방식이 잘 드러난다.

In [30]:
reset_graph()

n_inputs = 2
n_hidden1 = 3

original_w = [[1., 2., 3.], [4., 5., 6.]] # 다른 프레임워크로부터 가중치를 로드
original_b = [7., 8., 9.]                 # 다른 프레임워크로부터 편향을 로드

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
# [...] 모델의 나머지를 구성

# hidden1 변수의 할당 노드에 대한 핸들을 구한다
with tf.variable_scope("", default_name="", reuse=True):  # 루트 범위
    hidden1_weights = tf.get_variable("hidden1/kernel")
    hidden1_biases = tf.get_variable("hidden1/bias")

# 전용 플레이스홀더와 할당 노드를 만든다
original_weights = tf.placeholder(tf.float32, shape=(n_inputs, n_hidden1))
original_biases = tf.placeholder(tf.float32, shape=n_hidden1)
assign_hidden1_weights = tf.assign(hidden1_weights, original_weights)
assign_hidden1_biases = tf.assign(hidden1_biases, original_biases)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    sess.run(assign_hidden1_weights, feed_dict={original_weights: original_w})
    sess.run(assign_hidden1_biases, feed_dict={original_biases: original_b})
    # [...] 새 작업에 모델을 훈련시킨다
    print(hidden1.eval(feed_dict={X: [[10.0, 11.0]]}))

[[ 61.  83. 105.]]


### 11.2.3 신경망의 하위층을 학습에서 제외하기

신경망의 하위층은 저수준 특성을 감지하도록 학습되어서 다른 이미지를 분류할 때도 유용하다. 그래서 이 층들은 그대로 재사용할 수 있다.

일반적으로 새로운 DNN을 훈련시킬 때 재사용되는 층들의 가중치를 동결하는 것이 좋다. 하위층의 가중치가 고정되면 상위층의 가중치를 훈련시키기 쉽기 때문이다. (하위층의 출력은 상위층의 입력이 되므로 하위층의 가중치가 학습됨에 따라 변경되면 상위층으로의 입력이 계속 바뀌게 된다. 하위층의 가중치가 고정되면 학습이 반복되더라도 동일한 이미지에 대해서는 항상 같은 출력이 상위층으로 전달된다.)  훈련하는 동안 하위층을 고정시키는 한 가지 방법은 하위층의 변수를 제외하고 훈련시킬 변수 목록을 옵티마이저에 전달하는 것이다.

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # 재사용
n_hidden2 = 50  # 재사용
n_hidden3 = 50  # 재사용
n_hidden4 = 20  # 새로 만듦!
n_outputs = 10  # 새로 만듦!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # 재사용
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # 재사용
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # 재사용
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # 새로 만듦!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # 새로 만듦!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [0]:
with tf.name_scope("train"):                                        
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="hidden[34]|outputs")
training_op = optimizer.minimize(loss,var_list=train_vars)

In [0]:
init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

In [34]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # 정규 표현식
restore_saver = tf.train.Saver(reuse_vars) # 1-3층 복원

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 검증 세트 정확도: 0.8966
1 검증 세트 정확도: 0.93
2 검증 세트 정확도: 0.9404
3 검증 세트 정확도: 0.9442
4 검증 세트 정확도: 0.948
5 검증 세트 정확도: 0.951
6 검증 세트 정확도: 0.9508
7 검증 세트 정확도: 0.9532
8 검증 세트 정확도: 0.9554
9 검증 세트 정확도: 0.9566
10 검증 세트 정확도: 0.956
11 검증 세트 정확도: 0.9568
12 검증 세트 정확도: 0.9568
13 검증 세트 정확도: 0.9578
14 검증 세트 정확도: 0.9586
15 검증 세트 정확도: 0.9578
16 검증 세트 정확도: 0.9576
17 검증 세트 정확도: 0.9602
18 검증 세트 정확도: 0.9592
19 검증 세트 정확도: 0.9604


In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # 재사용
n_hidden2 = 50  # 재사용
n_hidden3 = 50  # 재사용
n_hidden4 = 20  # 새로 만듦!
n_outputs = 10  # 새로 만듦!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              name="hidden1") # 동결층 재사용
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              name="hidden2") # 동결층 재사용
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu,
                              name="hidden3") # 동결하지 않고 재사용
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu,
                              name="hidden4") # 새로 만듦!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # 새로 만듦!

그래프에 stop_gradient() 층을 추가하는 방법도 있다. 이렇게 하면 이 층 아래의 모든 층이 동결된다.

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [38]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # 정규 표현식
restore_saver = tf.train.Saver(reuse_vars) # 1-3층 복원

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 검증 세트 정확도: 0.9022
1 검증 세트 정확도: 0.9304
2 검증 세트 정확도: 0.943
3 검증 세트 정확도: 0.9476
4 검증 세트 정확도: 0.9514
5 검증 세트 정확도: 0.9524
6 검증 세트 정확도: 0.9524
7 검증 세트 정확도: 0.9556
8 검증 세트 정확도: 0.9552
9 검증 세트 정확도: 0.9562
10 검증 세트 정확도: 0.9572
11 검증 세트 정확도: 0.955
12 검증 세트 정확도: 0.9572
13 검증 세트 정확도: 0.958
14 검증 세트 정확도: 0.9582
15 검증 세트 정확도: 0.9576
16 검증 세트 정확도: 0.9566
17 검증 세트 정확도: 0.9576
18 검증 세트 정확도: 0.9592
19 검증 세트 정확도: 0.9582


훈련 코드는 이전과 동일하다.

### 11.2.4 동결된 층 캐싱하기

동결된 층은 변하지 않기 때문에 각 훈련 샘플에 대해 가장 위쪽의 동결된 층에서 나온 출력을 캐싱하는 것이 가능하다. 

In [39]:
import numpy as np

n_batches = len(X_train) //batch_size

with tf.Session() as sess:
  init.run()
  restore_saver.restore(sess,"./my_model_final.ckpt")

  h2_cache = sess.run(hidden2,feed_dict={X:X_train})

  for epoch in range(n_epochs):
    shuffled_idx = np.random.permutation(len(X_train))
    hidden2_batches = np.array_split(h2_cache[shuffled_idx],n_batches)
    y_batches = np.array_split(y_train[shuffled_idx],n_batches)
    for hidden2_batch,y_batch in zip(hidden2_batches,y_batches):
      sess.run(training_op,feed_dict={hidden2:hidden2_batch,y:y_batch})

  saver_path = saver.save(sess,"./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


두 번째 은닉층의 출력을 배치로 만들어 주입했다. 

### 11.2.5 상위층을 변경, 삭제, 대체하기

원본 모델의 상위층은 하위층보다는 덜 유용하다. 새로운 작업에서 필요한 고수준 특성은 원본 작업에서 유용했던 특성과는 많이 다르기 때문이다. 그래서 재사용할 적절한 층의 개수를 알아야 한다. 

먼저 복사한 모든 층을 동결한다. 그 다음에 모델을 훈련시키고 얼마나 성능이 나오는지 지켜본다. 그 후 가장 위쪽의 은닉층 한 개나 두 개의 동결을 해제해서 역전파로 가중치가 변경되게 하고 성능이 향상되는지 확인한다. 훈련 데이터가 많을수록 많은 층을 동결 해제할 수 있다. 

그래도 좋은 성능을 얻을 수 없고 훈련 데이터가 적다면 가장 위쪽의 은닉층(들)을 제거하고 남은 은닉층을 다시 모두 동결해본다. 재사용에 적절한 층의 개수를 찾을 때까지 반복한다. 

### 11.2.6 모델 저장소

당면한 문제와 비슷한 작업을 훈련시킨 신경망을 찾으려면 먼저 자기 자신의 모델 카탈로그를 체크해보고, 다음으로 모델 저장소에서 찾아보면 된다. 텐서플로는 깃허브에 자체 모델 저장소를 가지고 있으며, 이 안에는 가장 성능이 뛰어난 이미지 분류 모델들이 대부분 포함되어 있다.

### 11.2.7 비지도 사전훈련

레이블된 훈련 데이터가 많지 않은 복잡한 문제가 있는데, 비슷한 작업에 대해 훈련된 모델을 찾지 못했을 경우 먼저 더 많은 레이블된 훈련 데이터를 모아볼 수 있다. 하지만 이것이 너무 힘들고 비용이 많이 든다면 '비지도 사전훈련'을 해볼 수 있다. 레이블이 없는 훈련 데이터가 많다면 제한된 볼츠만 머신(RBM)이나 오토인코더 같은 비지도 특성 추출 알고리즘을 사용해 맨 하위층부터 위로 올라가면서 차례로 한 층씩 학습시킬 수 있다. 각 층은 훈련된 이전 층의 출력으로 훈련된다. 그리고 훈련 중인 층을 제외하고 다른 층은 모두 동결된다. 이런 방식으로 모든 층이 훈련되면 지도 학습으로(역전파 알고리즘을 사용해) 신경망을 세밀하게 튜닝할 수 있다.

### 11.2.8 보조 작업으로 사전훈련

레이블된 훈련 데이터가 많지 않은 복잡한 문제에서, 앞의 방법들을 사용할 수 없다면 레이블된 훈련 데이터를 쉽게 얻거나 생성할 수 있는 보조 작업에 첫 번째 신경망을 훈련시킬 수도 있다. 그리고 이 신경망의 하위층을 실제 작업을 위해 재사용한다. 첫 번째 신경망의 하위층은 두 번째 신경망에 재사용될 수 있는 특성 추출기를 학습하게 된다. 

예를 들어 얼굴을 인식하는 시스템을 만들고자 하는데 개인별 이미지가 얼마 없다면 좋은 분류기를 훈련시키기에 충분하지 않다. 그러나 인터넷에서 무작위로 많은 인물의 이미지를 수집해서 두 개의 다른 이미지가 같은 사람의 것인지 감지하는 첫 번째 신경망을 훈련시킬 수 있다. 이런 신경망은 얼굴의 특성을 잘 감지하도록 학습될 것이므로 이런 신경망의 하위층을 재사용한다면, 적은 양의 훈련 데이터로도 얼굴을 잘 구분하는 분류기를 훈련시킬 수 있다. 

레이블되지 않은 훈련 샘플은 레이블링에 비용이 많이 든다. 그래서 일반적으로 훈련 샘플 전체를 '좋은 샘플'로 레이블하고 이를 오염시켜 '나쁜 샘플'로 레이블하고는 한다. 다른 방식은 첫 번째 신경망이 각 훈련 샘플에 대해 점수를 출력하도록 훈련시키고 좋은 샘플의 점수가 나쁜 샘플의 점수보다 일정 마진 이상 더 크게 만드는 비용 함수를 사용하는 것이다. 이를 '최대 마진 학습'이라고 한다.

## **11.3 고속 옵티마이저**

지금까지 본 훈련 속도를 높이는 네 가지 방법은 연결 가중치에 좋은 초기화 전략 적용하기, 좋은 활성화 함수 사용하기, 배치 정규화 사용하기, 미리 훈련된 신경망의 일부 재사용하기이다. 또 다른 방법으로는 표준적인 경사 하강법 옵티마이저 대신 고속 옵티마이저를 사용하는 것이 있다.

### 11.3.1 모멘텀 최적화

볼링공이 매끈한 표면의 완만한 경사를 따라 굴러간다고 할 때, 이 공은 처음에는 느리게 출발하더라도 종단속도에 도달할 때까지 계속 가속된다. 이것이 모멘텀 최적화의 원리이다. 반면 표준적인 경사 하강법은 경사면을 따라 일정한 크기의 스텝으로 조금씩 내려간다. 그래서 맨 아래에 도착하는 데 더 오랜 시간이 걸린다. 
경사 하강법과 달리 모멘텀 최적화는 이전 그래디언트가 얼마였는지를 중요하게 새각한다. 매 반복에서 학습률을 곱한 현재 그래디언트를 모멘텀 벡터 m에 더하고 이 값을 빼는 방식으로 가중치를 갱신한다. 다시 말해 그래디언트를 속도가 아니라 가속도로 사용한다. 일종의 마찰저항을 표현하고 모멘텀이 너무 커지는 것을 막기 위해 이 알고리즘에는 모멘텀이라는 새로운 하이퍼파라미터 beta 가 등장한다. 

배치 정규화를 사용하지 않는 심층 신경망에서 상위층은 종종 스케일이 매우 다른 입력을 받게 되는데, 이 때 모멘텀 최적화를 사용하면 도움이 된다. 또한 지역 최적점을 건너뛰도록 하는 데도 도움이 된다. 

In [0]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9) #모멘텀 값은 일반적으로 0.9

텐서플로에서 모멘텀 최적화를 구현했다. 모멘텀 최적화를 이용하면 튜닝할 하이퍼파라미터가 하나 더 늘어난다는 단점이 있지만 경사 하강법보다 거의 항상 더 빠르다.

### 11.3.2 네스테로프 가속 경사

네스테로프 가속 경사(네스테로프 모멘텀 최적화)는 모멘텀 최적화의 한 변종으로, 거의 항상 기본 모멘텀 최적화보다 빠르다. 기본 아이디어는 현재 위치가 아니라 모멘텀 벡터의 방향으로 조금 앞서서 비용 함수의 그래디언트를 계산하는 것이다. 일반적으로 모멘텀 벡터가 올바른 방향, 즉 최적점을 향하는 방향을 가리킬 것이므로 원래 위치에서의 그래디언트를 사용하는 것보다 그 방향으로 조금 더 나아가서 측정한 그래디언트를 사용하는 것이 더 정확한 경우가 많다. 

In [0]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9,use_nesterov=True)

이처럼 기본 모멘텀 최적화를 구현하는 코드에 use_nesterov 인자를 추가하여 True로 설정해주기만 하면 된다.

### 11.3.3 AdaGrad

한쪽이 길쭉한 그릇 문제에서 경사 하강법은 가장 가파른 경사를 따라 빠르게 내려가기 시작해서 골짜기 아래로 느리게 이동한다. 알고리즘이 이를 일찍 감지하고 전역 최적점 쪽으로 더 정확한 방향을 잡으면 더 나을 것이다. AdaGrad 알고리즘(Adaptive Gradient Algorithm)은 가장 가파른 차원을 따라 그래디언트 벡터의 스케일을 감소시켜 이를 가능하게 한다. 이 알고리즘은 학습률을 감소시키지만 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소된다. 이를 적응적 하급률이라고 부르고 전역 최적점 방향으로 더 곧장 가도록 갱신되는 데 도움이 된다. 학습률 하이퍼파라미터 η(에타)를 덜 튜닝해도 되는 점이 또 하나의 장점이다. 

AdaGrad는 간단한 2차방정식 문제에 대해서는 잘 작동하지만 신경망을 훈련시킬 때 너무 일찍 멈춰버리는 경향이 있다. 학습률이 너무 감소되어 전역 최적점에 도착하기 전에 알고리즘이 완전히 멈춰버리는 것이다. 그래서 텐서플로에 AdagradOptimizer가 있지만 심층 신경망에는 사용하지 않고, 선형 회귀 같은 간단한 작업에 사용한다. 

### 11.3.4 RMSProp

AdaGrad는 너무 빠르게 느려져서 전역 최적점에 수렴하지 못하지만 RMSProp 알고리즘은 훈련 시작부터의 모든 그래디언트가 아닌 가장 최근 반복에서 비롯된 그래디언트만 누적함으로써 이 문제를 해결했다. 이렇게 하기 위해 알고리즘의 첫 번째 단계에서 지수 감소를 사용했다. 감쇠율 beta는 보통 0.9로 설정한다. beta 라는 하이퍼파라미터가 하나 더 생겼지만 대체로 기본값인 0.9로 잘 작동해 이를 튜닝할 필요는 없다. 

In [0]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.9,epsilon=1e-10)

텐서플로의 RMSPropOptimizer 파이썬 클래스를 이용했다. 아주 간단한 문제를 제외하고는 이 옵티마이저가 언제나 AdaGrad보다 성능이 좋다. 또 일반적으로 모멘텀 최적화나 네스테로프 가속 경사보다 더 빠르게 수렴하여 Adam 최적화 등장 전까지 연구자들이 가장 선호하는 최적화 알고리즘이었다.

### 11.3.5 Adam 최적화

적응적 모멘트 추정(adaptive moment estimation)을 의미하는 Adam은 모멘텀 최적화와 RMSProp의 아이디어를 합친 것이다. 모멘텀 최적화처럼 지난 그래디언트의 지수 감소 평균을 따르고 RMSProp 처럼 지난 그래디언트 제곱의 지수 감소된 평균을 따른다.

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

모멘텀 감쇠 하이퍼파라미터(beta 1)는 보통 0.9로 초기화하고 스케일 감쇠 하이퍼파라미터(beta 2)는 0.999로 초기화하는 경우가 많다. 또한 입실론은 보통 10의 -8승 같은 아주 작은 값으로 초기화한다. 이 값들이 텐서플로의 AdamOptimizer 파이썬 클래스의 기본값이다. 

Adam은 적응적 학습률 알고리즘이므로 학습률 하이퍼마라미터 η(에타)를 튜닝할 필요가 거의 없다. 기본값 η=0.001을 일반적으로 사용하게 된다. 

지금까지 논의한 최적화기법들은 1차 편미분(Jacobian)에만 의존한다. 최적화 이론에는 2차 편미분(Hessian)을 기반으로 한 뛰어난 알고리즘들이 있는데, 이는 심층 신경망에 적용하기는 어렵다. 

모든 최적화 알고리즘은 대부분의 파라미터가 0이 아닌 밀집 모델을 만든다. 그런데 엄청 빠르게 실행할 모델이나 메모리를 적게 차지하는 모델이 필요하면 희소 모델을 만들 수 있다. 첫 번째 방법은 모델을 훈련시킨 뒤 작은 값의 가중치를 0으로 만드는 것이고, 두 번째 방법은 훈련하는 동안 l1 규제를 강하게 적용하여 옵티마이저가 가능한 한 많은 가중치를 0으로 만들도록 강제하는 방법이다. 이런 기법이 충분하지 않을 때 쓰는 마지막 수단은 FTRL(Follow The Regularized Leader)이라는 기법으로, 쌍대 평균을 적용한다. 텐서플로는 FTRLOptimizer에 이 기법의 변종인 FTRL-Proximal 알고리즘을 구현했다. 


### 11.3.6 학습률 스케일링

학습률을 너무 크게 잡으면 훈련이 발산할 수 있고, 학습률을 너무 작게 잡으면 최적점에 수렴할 수는 있지만 시간이 많이 걸리기 때문에 좋은 학습률을 찾는 것이 중요하다. 최적의 고정 학습률을 찾을 수도 있지만, 이보다 높은 학습률로 시작해 학습 속도가 느려질 때 학습률을 낮추는 방식이 좋은 솔루션에 더 빨리 도달하게 할 수 있다. 훈련하는 동안 학습률을 감소시키는 전략을 '학습 스케줄'이라고 하며, 가장 보편적인 것에는 다음 네 가지가 있다.

1. 미리 정의된 개별적인 고정 학습률
처음에 특정 학습률을 지정하고, 특정 에포크 후에 더 작은 학습률로 바꾼다. 적절한 학습률과 적당한 시점을 찾으려면 여러 번 바꿔봐야할 수도 있다.

2. 성능 기반 스케줄링
매 N 스텝마다 검증 오차를 측정하고 오차가 줄어들지 않으면 lambda만큼 학습률을 감소시킨다. (조기종료와 비슷한 느낌이라고 생각하면 됨)

3. 지수 기반 스케줄링
반복 횟수 t의 함수 η(t) = η0*(10의 -t/r 승) 로 학습률을 설정한다. 에타0과 r을 튜닝해줘야한다. 

4. 거듭제곱 기반 스케줄링
학습률을 η(t) = η0*((1+t/r)의 -c 승) 으로 설정한다. 하이퍼파라미터 c는 보통 1로 지정된다. 지수 기반 스케줄링과 비슷하지만 학습률이 더 느리게 감소된다. c는 1일 때의 거듭제곱 기반 스케줄링 함수가 텐서플로의 inverse_time_decay() 함수에 구현되어있다.


In [0]:
initial_learning_rate = 0.1
decay_steps = 10000
decay_rate = 1/10
global_step = tf.Variable(0,trainable=False,name="global_step")
learning_rate = tf.train.exponential_decay(initial_learning_rate,global_step,decay_steps,decay_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate,momentum=0.9)
trainig_op = optimizer.minimize(loss, global_step=global_step)

텐서플로에서 학습 스케줄을 구현했다.

하이퍼파라미터 값을 지정한 후 현재 훈련 반복 횟수를 저장하되 학습되지 않는 변수 global_step(초깃값은 0)을 생성했다. exponential_decay() 함수로 지수 감소 학습률을 정의했다. 그리고 이 감소 학습률로 옵티마이저를 만들었다. 마지막으로 옵티마이저의 minimize() 메서드를 호출해 훈련 연산을 생성했다. 

## **11.4 과대적합을 피하기 위한 규제 방법**

심층 신경망은 수만 개, 때로는 수백만 개의 파라미터를 가지고 있다. 파라미터가 많아서 네트워크의 자유도가 높고 크고 복잡한 데이터셋을 학습할 수 있지만, 훈련 세트에 과대적합되기 쉽다는 것을 의미하기도 한다.

### 11.4.1 조기종료

훈련 세트에 과대적합되는 것을 피하기 위한 좋은 방법 중 하나는 조기종료이다. 검증 세트의 성능이 떨어지기 시작할 때 훈련을 중지시키면 된다. 

### 11.4.2 $\ell_1$과 $\ell_2$ 규제

$\ell_1$과 $\ell_2$ 규제를 사용해 신경망의 연결 가중치에 제약을 가할 수 있다. 일반적으로 편향에는 적용하지 않는다. 텐서플로를 사용해 구현하려면 비용 함수에 적절한 규제항을 추가하면 된다.

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")

먼저 모델을 만들었다. 간단하게 하기 위해 은닉층은 하나만 만들었다.

In [0]:
W1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0") #은닉층의 가중치를 W1로 설정했다.
W2 = tf.get_default_graph().get_tensor_by_name("outputs/kernel:0") #출력층의 가중치를 W2로 설정했다.

scale = 0.001 # l1 규제 하이퍼파라미터

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2))
    loss = tf.add(base_loss, scale * reg_losses, name="loss")

그 다음, 층의 가중치에 대한 핸들을 얻어 크로스 엔트로피 손실에 $\ell_1$ 손실(즉, 가중치의 절댓값)을 더해 전체 손실을 계산했다.

In [0]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [49]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.831
1 검증 세트 정확도: 0.871
2 검증 세트 정확도: 0.8838
3 검증 세트 정확도: 0.8934
4 검증 세트 정확도: 0.8966
5 검증 세트 정확도: 0.8988
6 검증 세트 정확도: 0.9016
7 검증 세트 정확도: 0.9044
8 검증 세트 정확도: 0.9058
9 검증 세트 정확도: 0.906
10 검증 세트 정확도: 0.9068
11 검증 세트 정확도: 0.9054
12 검증 세트 정확도: 0.907
13 검증 세트 정확도: 0.9084
14 검증 세트 정확도: 0.9088
15 검증 세트 정확도: 0.9064
16 검증 세트 정확도: 0.9066
17 검증 세트 정확도: 0.9066
18 검증 세트 정확도: 0.9066
19 검증 세트 정확도: 0.9052


층이 많으면 이런 방식은 번거로울 수 있다. 다른 방법으로는 tf.layers.dense() 함수에 규제 함수를 전달할 수 있다. 이 함수는 규제 손실을 계산하기 위한 연산을 만들고 규제 손실 컬렉션에 이 연산을 추가한다.

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
scale = 0.001


my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None,
                            name="outputs")

동일한 매개변수를 매번 반복하지 않으려고 파이썬의 partial() 함수를 사용했다. kernel_regularizer 매개변수를 지정해주었다. 가중치를 매개변수로 받아 규제에 상응하는 손실을 반환하는 함수를 이 매개변수에 전달할 수 있다. 위에서는 l1_regularizer() 함수를 전달했다.

두 개의 은닉층과 한 개의 출력층으로 이루어진 신경망을 만들고, 각 층의 가중치에 상응하는 $\ell_1$ 규제 손실을 계산하기 위한 노드도 그래프에 추가했다. 

In [0]:
with tf.name_scope("loss"):                                     
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  
        labels=y, logits=logits)                                
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")

전체 손실에 규제 손실을 추가해주었다.

### 11.4.3 드롭아웃

심층 신경망에서 가장 인기 있는 규제 방법은 드롭아웃(제외)이다. 

매 훈련 스텝에서 각 뉴런은 임시적으로 드롭아웃될 확률 p를 가진다. 즉 이번 훈련 스텝에서 완전히 무시되더라도 다음 스텝에서 활성화될 수 있다. 하이퍼파라미터 p를 드롭아웃 비율이라고 하며 보통 50%로 지정한다. 훈련이 끝난 후에는 뉴런에 더 이상 드롭아웃을 적용하지 않는다. 

드롭아웃으로 훈련된 뉴런은 이웃한 뉴런에 맞추어 적응될 수 없다. 가능한 한 자기 자신이 유용해져야한다는 뜻이다. 또 이런 뉴런들은 몇 개의 입력 뉴런에만 지나치게 의존할 수 없다. 모든 입력 뉴런에 주의를 기울여야 하기 때문에 입력값의 작은 변화에 덜 민감해지게 된다. 결국 더 안정적인 네트워크가 되어 일반화 성능이 좋아진다. 

드롭아웃의 능력을 이해하는 또 다른 방법은 각 훈련 스텝에서 고유한 네트워크가 생성된다고 생각하는 것이다. 드롭아웃이 가능한 뉴런 수를 N이라고 했을 때, 개개의 뉴런이 있을 수도 없을 수도 있기 때문에 2의 N승 만큼의 네트워크가 가능하다. 10,000번의 훈련 스텝을 진행하면 10,000개의 다른 신경망을 훈련시키게 된다. 이 신경망은 대부분의 가중치를 공유하고 있기 때문에 독립적이라고 볼 수는 없지만 모두 다르다. 결과적으로 만들어진 신경망은 이 모든 신경망을 평균한 앙상블로 볼 수 있다.

p=0.5 라 하면 테스트하는 동안 한 뉴런이 훈련 때보다 평균적으로 두 배 많은 입력 뉴런과 연결되기 때문에 훈련하고 나서 각 뉴런의 연결 가중치에 0.5를 곱해줄 필요가 있다. (곱하기 2가 된 것을 다시 나누기 2해주어 보상해준다고 보면 됨) 일반화하면 훈련이 끝난 뒤 각 입력의 연결 가중치에 보존 확률(1-p)을 곱해야 한다. 

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [54]:
training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                              name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                              name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

Instructions for updating:
Use keras.layers.dropout instead.


텐서플로로 세 개의 층으로 된 신경망에 드롭아웃 규제를 적용했다. 드롭아웃을 구현하기 위해서는 tf.layers.dropout() 함수를 입력층이나 원하는 은닉층의 출력에 적용하기만 하면 된다. 훈련하는 동안 이 함수는 무작위로 뉴런을 제거하며 나머지 출력을 보존 확률로 나눈다. 

모델이 과대적합된 것 같으면 드롭아웃 비율을 올리고, 반대로 과소적합인 경우 드롭아웃 비율을 낮춰야 한다. 또한 네트워크에 층이 많을 경우에는 드롭아웃 비율을 높이고 소규모 네트워크에서는 줄이는 것이 좋다.

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [56]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.9246
1 검증 세트 정확도: 0.9468
2 검증 세트 정확도: 0.9492
3 검증 세트 정확도: 0.9508
4 검증 세트 정확도: 0.9628
5 검증 세트 정확도: 0.9632
6 검증 세트 정확도: 0.9618
7 검증 세트 정확도: 0.9664
8 검증 세트 정확도: 0.967
9 검증 세트 정확도: 0.967
10 검증 세트 정확도: 0.9684
11 검증 세트 정확도: 0.9686
12 검증 세트 정확도: 0.9688
13 검증 세트 정확도: 0.97
14 검증 세트 정확도: 0.972
15 검증 세트 정확도: 0.9712
16 검증 세트 정확도: 0.9728
17 검증 세트 정확도: 0.9734
18 검증 세트 정확도: 0.9738
19 검증 세트 정확도: 0.9732


### 11.4.4 맥스-노름 규제 

신경망에서 아주 널리 사용되는 또 다른 규제 기법은 맥스-노름 규제이다. 맥스-노름 규제는 배치 정규화를 사용하지 않았을 때 그래디언트 감소/폭주 문제를 완화하는 데 도움을 줄 수 있다.

In [0]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

두 개의 은닉층을 가진 MNIST 신경망을 만들었다.

In [58]:
threshold = 1.0
weights = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
clipped_weights = tf.clip_by_norm(weights,clip_norm=threshold,axes=1)
clip_weights = tf.assign(weights,clipped_weights)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


첫 번째 은닉층의 가중치를 구한 뒤 clip_by_norm() 함수를 이용해 두 번째 축을 따라(2차원 텐서에서 첫 번째 축은 위에서 아래 방향을, 두 번째 축은 왼쪽에서 오른쪽 방향을 나타냄) 각 행 벡터의 최대 norm이 1.0이 되도록 가중치를 클리핑하는 연산을 만들었다.

In [0]:
weights2 = tf.get_default_graph().get_tensor_by_name("hidden2/kernel:0")
clipped_weights2 = tf.clip_by_norm(weights2, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights2, clipped_weights2)

두 번째 층에 대해서도 동일하게 해주었다.

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

초기화 연산과 Saver 객체를 만들었다.

In [0]:
n_epochs = 20
batch_size = 50

In [62]:
with tf.Session() as sess:                                              
    init.run()   
    for epoch in range(n_epochs):                                                       
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights.eval()
            clip_weights2.eval()                                        
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid}) 
        print(epoch, "검증 세트 정확도:", accuracy_val)                     

    save_path = saver.save(sess, "./my_model_final.ckpt")  

0 검증 세트 정확도: 0.9568
1 검증 세트 정확도: 0.9696
2 검증 세트 정확도: 0.971
3 검증 세트 정확도: 0.9776
4 검증 세트 정확도: 0.977
5 검증 세트 정확도: 0.9782
6 검증 세트 정확도: 0.982
7 검증 세트 정확도: 0.982
8 검증 세트 정확도: 0.9794
9 검증 세트 정확도: 0.9818
10 검증 세트 정확도: 0.9826
11 검증 세트 정확도: 0.9846
12 검증 세트 정확도: 0.9818
13 검증 세트 정확도: 0.9836
14 검증 세트 정확도: 0.9844
15 검증 세트 정확도: 0.9842
16 검증 세트 정확도: 0.984
17 검증 세트 정확도: 0.9842
18 검증 세트 정확도: 0.9842
19 검증 세트 정확도: 0.9846


모델을 훈련시켰다. 이전과 매우 유사한데 training_op을 실행한 후에 clip_weights와 clip_weights2 연산을 실행하는 것만 다르다.

그런데 이 클리핑을 모든 은닉층에 적용해야 한다는 점에서 조금 번거롭다. 깔끔하게 하고 싶다면 max_norm_regularizer() 함수를 만들어 앞서 보았떤 l1_regularizer() 함수와 같이 사용하면 된다.

In [0]:
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # 규제 손실을 위한 항이 없다
    return max_norm

In [0]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
max_norm_reg = max_norm_regularizer(threshold=1.0)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 20
batch_size = 50

In [69]:
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "검증 세트 정확도:", accuracy_val)                      

    save_path = saver.save(sess, "./my_model_final.ckpt") 

0 검증 세트 정확도: 0.9558
1 검증 세트 정확도: 0.97
2 검증 세트 정확도: 0.9732
3 검증 세트 정확도: 0.9748
4 검증 세트 정확도: 0.9754
5 검증 세트 정확도: 0.979
6 검증 세트 정확도: 0.981
7 검증 세트 정확도: 0.9812
8 검증 세트 정확도: 0.9814
9 검증 세트 정확도: 0.9806
10 검증 세트 정확도: 0.9822
11 검증 세트 정확도: 0.9816
12 검증 세트 정확도: 0.9812
13 검증 세트 정확도: 0.9822
14 검증 세트 정확도: 0.9826
15 검증 세트 정확도: 0.9824
16 검증 세트 정확도: 0.982
17 검증 세트 정확도: 0.9824
18 검증 세트 정확도: 0.9826
19 검증 세트 정확도: 0.9824


In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 20
batch_size = 50

In [72]:
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid}) 
        print(epoch, "검증 세트 정확도:", accuracy_val)                     

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 검증 세트 정확도: 0.9554
1 검증 세트 정확도: 0.9698
2 검증 세트 정확도: 0.9734
3 검증 세트 정확도: 0.9726
4 검증 세트 정확도: 0.976
5 검증 세트 정확도: 0.977
6 검증 세트 정확도: 0.9798
7 검증 세트 정확도: 0.9822
8 검증 세트 정확도: 0.9814
9 검증 세트 정확도: 0.9762
10 검증 세트 정확도: 0.9824
11 검증 세트 정확도: 0.984
12 검증 세트 정확도: 0.9832
13 검증 세트 정확도: 0.984
14 검증 세트 정확도: 0.9834
15 검증 세트 정확도: 0.9844
16 검증 세트 정확도: 0.9836
17 검증 세트 정확도: 0.985
18 검증 세트 정확도: 0.9842
19 검증 세트 정확도: 0.9844


### 11.4.5 데이터 증식

데이터 증식은 기존의 데이터에서 새로운 데이터를 생성해 인공적으로 훈련 세트의 크기를 늘린다. 이 방식은 과대적합을 줄이므로 규제의 방도로 사용된다. 이 기법은 실제와 같은 훈련 샘플을 생성하는 것으로, 단순히 백색소음(적용한 수정사항이 학습 가능한 것이 아님)을 추가하는 것은 도움이 되지 않는다. 

예를 들어 버섯 이미지를 구분하는 모델이 있다면 훈련 세트에 있는 모든 이미지를 다양하게 조금씩 이동, 회전하거나 크기를 바꿔서 만든 이미지를 훈련 세트에 추가한다. 이렇게 하면 모델이 사진에 있는 버섯의 위치, 각도, 크기에 덜 민감해진다. 

텐서플로는 밝기, 명암, 채도, 색조 조정, 위치 변경, 회전, 크기 조절, 뒤집기, 자르기 등과 같은 다양한 이미지 조작 연산을 제공한다. 이런 연산들을 사용하면 이미지 데이터셋에서 데이터 증식을 손쉽게 구현할 수 있다.